In [9]:
# load the necessary functions
for (fnm in list.files("/home/jovyan/shared_materials/rwwa", full.names = T)) source(fnm)
library(extRemes)

Loading required package: Lmoments

Loading required package: distillery


Attaching package: ‘extRemes’


The following objects are masked from ‘package:stats’:

    qqnorm, qqplot




# Specify the return period of the event

In [10]:
rp <- 20

# Fit the same statistical model to all CORDEX time series

In [11]:
fl = list.files(".", pattern = "ts-annual_tasmax_EUR-11", full.names = T)

In [13]:
for(fnm in fl) {

    # if your file name has a different format, you'll need to change these expressions
    gcm <- strsplit(fnm, "_")[[1]][4]
    rcm <- strsplit(fnm, "_")[[1]][5]

    # construct the new filename
    res_fnm <- paste0("res-cordex_tasmax_EUR-11_",gcm,"_",rcm)

    # if the file hasn't already been processed...
    if(!file.exists(res_fnm)) {

        # get the filename of the GCM's GMST
        gmst_fnm <- list.files("/home/jovyan/shared_materials/CMIP5_GSAT", pattern = gcm, full.names = T)

        # make sure the file name has picked up correctly - if not, skip
        if (length(gmst_fnm) == 0) { next }

        # load the data
        df <- merge(load_ts(gmst_fnm, col.names = c("year", "gmst")),
                    read.csv(fnm)[,c("year", "tasmax")])       # rename the columns if necessary

        # fit the same statistical model that you used for the observations
        mdl <- fit_ns(dist = "gev", type = "shift", data = df, varnm = "tasmax", covnm = "gmst", lower = F)
        
        # use model's 2025 GMST to define factual climate        
        cov_now <- df[df$year == 2025,"gmst",drop = F]

        # we want a historic counterfactual climate that's 1.3C cooler, and a future that's 1.3C warmer
        cov_hist <- cov_now - 1.3
        cov_fut <- cov_now + 1.3

        # bootstrap results
        #   y_now should be the current (factual) year
        #   y_start should correspond to the first year of your observations
        #   y_fut determines how far into the future to look - no further than 2070
        #   if using precip (relative changes in intensity), set `di_relative = T` - otherwise leave as F
        res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut, 
                              y_now = 2025, y_start = 1940, y_fut = 2070, di_relative = F)

        # save the results as a data.frame
        write.csv(res, res_fnm)
        
        # quick return level plots to check fit
        rlplot_fnm <- gsub("csv", "png", paste0("mdl-eval_", res_fnm))
        if(!file.exists(rlplot_fnm)) {
            png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
                plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, ev = res[,"rp_value"], nsamp = 100, main = strsplit(fnm, "/")[[1]])
            }; dev.off()
        }
    }
}

## Compile all the model results into one .csv

In [14]:
# load all results and compile into a single data.frame
res <- t(sapply(list.files(".", pattern = "^res-cordex_tasmax_EUR-11", full.names = T), read.csv, row.names = "X"))

# relabel the rows with just the model name
rownames(res) <- sapply(rownames(res), function(r) gsub(".csv","",paste0(strsplit(r, "_")[[1]][c(4:5)], collapse = "_")))

# check that the rownames are right
unname(rownames(res))

[1] "CCCma-CanESM2_CLMcom-CCLM4-8-17"     "CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63"
[3] "IPSL-IPSL-CM5A-MR_DMI-HIRHAM5"       "MOHC-HadGEM2-ES_KNMI-RACMO22E"      
[5] "MOHC-HadGEM2-ES_MOHC-HadREM3-GA7-05" "MPI-M-MPI-ESM-LR_DMI-HIRHAM5"       
[7] "MPI-M-MPI-ESM-LR_MPI-CSC-REMO2009"   "NCC-NorESM1-M_SMHI-RCA4"

In [15]:
# save as a .csv
write.csv(res, "res-models_cordex.csv")

## Compile all the observational results into one .csv

In [16]:
# loop over the observational results, unfold the data.frame of results
obs_res <- t(sapply(list.files(".", pattern = "^res-obs", full.names = T), function (fnm) {
    res <- read.csv(fnm, row.names = "X")
    unlist(lapply(rownames(res), function(cnm) setNames(res[cnm,], paste(gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
}))

# clean up the row names (you could set these manually if preferred)
rownames(obs_res) <- sapply(rownames(obs_res), function(r) gsub(".csv","",strsplit(r, "_")[[1]][2]))
write.csv(obs_res, "res-obs.csv")

# Next steps

Take a copy of the results spreadsheet: https://docs.google.com/spreadsheets/d/1jMRnVECUbFKjCR-ueMXEzrqM8oKHQmOQL6HCrdMBiPI/edit?gid=1887990833#gid=1887990833

Copy and paste the results from the obs and models into the spreadsheet

Model evaluation
- do the models capture the spatial pattern of the relevant index?
- do the models capture the seasonal cycle of the relevant index?
- do the ranges of parameter estimates from the models overlap with the ranges from the observations?
- decide which models will be included in the final synthesis

**Next: synthesise the results from the observations and selected climate models**